Copyright (c) 2024 Houssem Ben Braiek, Emilio Rivera-Landos, IVADO, SEMLA

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# The Modelling Process
In this notebook, we also leverage our library to automate repetitive tasks, ensuring a more efficient and organized workflow.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

# TODO: comment out if you want to check info logs
# import logging
# logging.basicConfig(level=logging.INFO)
import joblib
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from bank_marketing.data.make_datasets import make_bank_marketing_dataframe
from bank_marketing.data.prep_datasets import prepare_binary_classfication_tabular_data
from bank_marketing.features.skl_build_features import FeatureNames, make_data_transformer
from bank_marketing.models.skl_train_models import (
    accuracy_evaluation,
    print_accuracy_results,
    train_and_evaluate,
)

### Load the raw data saved in the EDA notebook

In [ ]:
df = make_bank_marketing_dataframe(
    '../data/start.db', '../data/external/socio_economic_indices_data.csv'
)

In [ ]:
columns = [
    'age',
    'job',
    'marital',
    'education',
    'comm_month',
    'comm_day',
    'comm_type',
    'comm_duration',
    'curr_n_contact',
    'days_since_last_campaign',
    'last_n_contact',
    'last_outcome',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed',
    'housing',
    'loan',
    'default',
    'curr_outcome',
]

In [ ]:
df = df[columns]

## Iteration 1
In this initial iteration, we prepare the dataset for binary classification modeling and proceed to test our first baseline model, the Decision Tree, using our library.

In [ ]:
predictors = list(df.columns)
predictors.remove('curr_outcome')
predictors.remove('comm_duration')
predicted = 'curr_outcome'

In [ ]:
dataset = prepare_binary_classfication_tabular_data(
    df, predictors, predicted, pos_neg_pair=('yes', 'no')
)

In [ ]:
numerical_cols = dataset.train_x.select_dtypes(exclude='object').columns.tolist()
categorical_cols = dataset.train_x.select_dtypes('object').columns.tolist()
feature_names = FeatureNames(numerical_cols, categorical_cols)
input_transformer = make_data_transformer(feature_names)
classifier = DecisionTreeClassifier()

In [ ]:
model = Pipeline(
    [
        ('input_transformer', input_transformer),
        ('classifier', classifier),
    ]
)
model.fit(dataset.train_x, dataset.train_y)
results = accuracy_evaluation(model, dataset)
print_accuracy_results(results)

In [ ]:
RocCurveDisplay.from_predictions(dataset.test_y.values, model.predict_proba(dataset.test_x)[:, 1])

## Iteration 2:
The second iteration is about testing multiple models and compare their performance. 

In [ ]:
classifiers_list = [DecisionTreeClassifier, MLPClassifier, RandomForestClassifier]
results = train_and_evaluate(dataset, feature_names, classifiers_list)
print_accuracy_results(results)

## Iteration 3
The third iteration is about testing advanced feature engineering with multiple models and comparing their performance. 

In [ ]:
from bank_marketing.features.skl_build_features import (
    AdvFeatureNames,
    make_advanced_data_transformer,
)
from bank_marketing.models.skl_train_models import advanced_train_and_evaluate

In [ ]:
person_info_cols_cat = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'comm_type']
person_info_cols_num = ['age']
num_cols_wo_customer = [
    'curr_n_contact',
    'days_since_last_campaign',
    'last_n_contact',
    'emp.var.rate',
    'cons.price.idx',
    'cons.conf.idx',
    'euribor3m',
    'nr.employed',
]
cat_cols_wo_customer = ['comm_month', 'comm_day', 'last_outcome']

In [ ]:
adv_feature_names = AdvFeatureNames(
    person_info_cols_num, person_info_cols_cat, num_cols_wo_customer, cat_cols_wo_customer
)

In [ ]:
classifiers_list = [DecisionTreeClassifier, RandomForestClassifier]
clustering_algos_list = [KMeans, MiniBatchKMeans]
results = advanced_train_and_evaluate(
    dataset, adv_feature_names, clustering_algos_list, classifiers_list
)
print_accuracy_results(results)

In [ ]:
data_transformer = make_advanced_data_transformer(adv_feature_names, MiniBatchKMeans)
classifier = RandomForestClassifier()
adv_model = Pipeline(
    [
        ('input_transformer', data_transformer),
        ('classifier', classifier),
    ]
)
adv_model.fit(dataset.train_x, dataset.train_y)
results = accuracy_evaluation(adv_model, dataset)
print_accuracy_results(results)

## Final Step
It is important now to save our trained model as a binary file for serving. The code is provided in the next cell.

In [ ]:
Path('../models/').mkdir(parents=True, exist_ok=True)
joblib.dump(adv_model, '../models/rf_with_minibatch_kmeans.joblib')

In [ ]:
# Let's load and test our model
loaded_adv_model = joblib.load('../models/rf_with_minibatch_kmeans.joblib')
results = accuracy_evaluation(loaded_adv_model, dataset)
print_accuracy_results(results)